# Работа с изображениями и сверточные сети

> 🚀 В этой практике нам понадобятся: `numpy==1.26.4, seaborn==0.13.2, matplotlib==3.8.4, scikit-learn==1.4.2, torch==2.1.2, opencv-python==4.9.0.80` 

> 🚀 Установить вы их можете с помощью команды: `%pip install numpy==1.26.4 seaborn==0.13.2 matplotlib==3.8.4 scikit-learn==1.4.2 torch==2.1.2 opencv-python==4.9.0.80` 


## Содержание

* [Загружаем данные с Kaggle](#Загружаем-данные-с-Kaggle)
* [Первое знакомство с данными](#Первое-знакомство-с-данными)
* [Вводная по работе с изображениями](#Вводная-по-работе-с-изображениями)
* [Смотрим данные](#Смотрим-данные)
  * [Задание - более широкий взгляд](#Задание---более-широкий-взгляд)
* [Подготовка изображений](#Подготовка-изображений)
* [Датасет в формате PyTorch](#Датасет-в-формате-PyTorch)
  * [Задание - пишем датасет изображений](#Задание---пишем-датасет-изображений)
* [Из чего состоит свёрточная нейросеть](#Из-чего-состоит-свёрточная-нейросеть)
  * [Операция свертки](#Операция-свертки)
    * [Паддинг (отступы)](#Паддинг-отступы)
    * [Stride (шаг свёртки)](#Stride-шаг-свёртки)
    * [Dilation (разреженность)](#Dilation-разреженность)
  * [Свётка в PyTorch](#Свётка-в-PyTorch)
  * [Нелинейность для свертки](#Нелинейность-для-свертки)
  * [MaxPooling](#MaxPooling)
* [Составим нашу первую сверточную сеть!](#Составим-нашу-первую-сверточную-сеть)
* [Функция потерь](#Функция-потерь)
* [Пора учить!](#Пора-учить)
* [Применение аппаратного ускорения для обучения и исполнения](#Применение-аппаратного-ускорения-для-обучения-и-исполнения)
* [А как оценивать то?](#А-как-оценивать-то?)
* [Пора учить по полной](#Пора-учить-по-полной)
* [Оценка модели на тесте, "слоеном тесте" =)](#Оценка-модели-на-тесте,-слоеном-тесте-=)
* [Анализ ошибок (вместе с визуальной оценкой)](#Анализ-ошибок-вместе-с-визуальной-оценкой)
* [Что мне делать со своими картинками?](#Что-мне-делать-со-своими-картинками?)
* [Рекомендации](#Рекомендации)
* [Вопросы (заметки)](#Вопросы-заметки)


В этом ноутбуке для вас откроются тайны, которые скрывают "сверточные сети". Они, как известно, отлично обрабатывают изображения, чтобы находить на них людей, определять количество машин и решать многие другие интересные задачки. 

In [ ]:
# Импорт необходимых модулей 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
from torch import nn
import typing

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = "black"

plt.rcParams["figure.figsize"] = (15, 10)
plt.rcParams["text.color"] = "black"
plt.rcParams["font.size"] = 14
plt.rcParams["axes.labelcolor"] = TEXT_COLOR
plt.rcParams["xtick.color"] = TEXT_COLOR
plt.rcParams["ytick.color"] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

## Загружаем данные с Kaggle

Работа с табличными данными на Kaggle может сводиться к тому, что вы просто скачиваете нужные файлы с их сайта и кладёте себе для работы. Когда нужно работать с большими файлами или множеством файлов, то можно воспользоваться их API. То есть вы можете сами скачать данные с Kaggle и положить в нужную папку программно!

Датасет, с которым мы будем работать, называется [MNIST as .jpg](https://www.kaggle.com/scolianni/mnistasjpg).

Первое, что нужно сделать - установить пакет kaggle с PyPI, чтобы можно было пользоваться их утилитами:

In [ ]:
%pip install -q kaggle==1.6.12

После того, как пакет установлен, нам нужно зайти в свой аккаунт на Kaggle (в настройки аккаунта) и получить файл `kaggle.json` с сайта. Его можно получить на машину переходом в настройки аккаунта и там кликом на кнопку "Create New API Token":

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/kaggle_create_api.png" width=600/></p>

Это выгрузит на ваш компьютер нужный файл. Положите его рядом с ноутбуком.

> 🔥 Очень важно, API Token - это по сути прямо доступ для управления аккаунтом. Токен с кагла или других сервисов ни в коем случае нельзя грузить в репозитории или открытые источники. Его нужно хранить в закрытом хранилище и проще сгенерировать новый, чем получить ситуацию, когда кто-то завладел вашим токеном.

Если вы работаете в Google Colab, то нужно загрузить файл токена на сервер.
> Специфика здесь в том, что все вычисления Google Colab производит у себя на серверах, поэтому, если хочется поработать в файлами, то их надо загрузить на этот сервер.

Сделать это можно нажатием кнопки "Загрузить в сессионное хранилище" в меню слева:

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/colab_upload.png" width=600/></p>

Или программно:
```
from google.colab import files
files.upload()
```

Этот код вызовет окошечко загрузки, где можно указать, какой файл загрузить.

> Обратите внимание, что перезапуск среды Google Colab может стереть все загруженные файлы, поэтому нужно загружать этот файл каждый раз, когда хотите загрузить данные с Kaggle в сессию Google Colab.

А теперь пора скачивать наш набор данных - MNIST:

In [ ]:
# Указываем, где лежит kaggle.json и скачиваем датасет
import os
os.environ["KAGGLE_CONFIG_DIR"] = os.curdir

In [ ]:
%%bash
kaggle datasets download -d scolianni/mnistasjpg

В результате мы можем увидеть архив в нашей папке `mnistasjpg.zip`. Самое время его распаковать и посмотреть, что там:

In [ ]:
# Создадим папку для набора данных и разархивируем в неё файлы из архива
import zipfile
DATASET_DIRECTORY="mnist_dataset"
os.makedirs(DATASET_DIRECTORY, exist_ok=True)

with zipfile.ZipFile("mnistasjpg.zip", "r") as zip_ref:
    zip_ref.extractall(DATASET_DIRECTORY)

print(os.listdir(DATASET_DIRECTORY))

Отлично, мы видим данные в виде папок и архивов. Описание мы взяли со страницы датасета:

This dataset is composed of four files:

- `trainingSet.tar.gz` (10.2 MB) - This file contains ten sub folders labeled 0 to 9. Each of the sub folders contains .jpg images from the Digit Recognizer competition"s train.csv dataset, corresponding to the folder name (ie. folder 2 contains images of 2"s, etc.). In total, there are 42,000 images in the training set.
- `testSet.tar.gz` (6.8 MB) - This file contains the .jpg images from the Digit Recognizer competition"s test.csv dataset. In total, there are 28,000 images in the test set.
- `trainingSample.zip` (407 KB) - This file contains ten sub folders labeled 0 to 9. Each sub folder contains 60 .jpg images from the training set, for a total of 600 images.
- `testSample.zip` (233 KB) - This file contains a 350 image sample from the test set.

По сути, у нас есть набор данных для обучения с разметкой `trainingSet`, набор данных для тестирования `testSet`, но он не имеет разметки. Это набор для проведения предсказания и загрузки на сервис kaggle для соревнований. Поэтому, `testingSet` мы будем использовать для визуальной проверки. Для оценки метрик нам придется взять `trainingSet` и разделить на нужные нам выборки.

Отлично, пора познакомиться с данными!

## Первое знакомство с данными

При работе с изображениями важно всегда делать визуальную оценку. Это позволяет заметить особенности в данных до принятия важных решения.

> ⚠️ Ну и не забываем одно из самых важных правил - чем лучше мы понимаем данные, тем лучше мы их можем обработать и улучшить качество работы модели!

Начнём с того, что просмотрим структуры папок (можно сделать это глазами в проводнике или командами из ноутбука).

В директории trainingSet лежат директории с номерами. Это по сути и есть разметка - таким образом указано, что в директории "0" лежат изображения с цифрой 0. Проверим эту гипотезу:

In [ ]:
# Очень удобная бибилотека для работы с путями в файловой системе
from pathlib import Path
import os

training_set_dirpath = Path("mnist_dataset") / "trainingSet" / "trainingSet"

directories_list = os.listdir(training_set_dirpath)
print(training_set_dirpath, directories_list)

In [ ]:
image_path = training_set_dirpath / "0" / "img_4.jpg"
image_path

Путь до изображения мы сделали, теперь, чтобы прочитать его, воспользуемся фреймворком OpenCV и его функцией [cv2.imread()](https://docs.opencv.org/4.5.3/d4/da8/group__imgcodecs.html#ga288b8b3da0892bd651fce07b3bbd3a56):

In [ ]:
# Используем библиотеку OpenCV для работы с изображениями
import cv2

image = cv2.imread(str(image_path))

plt.figure(figsize=(7,7))
plt.imshow(image)
plt.show()

Смотрите, действительно, на изображении цифра 0. Так мы убедились в правильной разметке одного файла, а еще 41999 файл =)

Нет, просматривать все не нужно, в том числе для этого мы и будем делать модель для классификации изображений.

> ⚠️ Одна из целей разработки модели машинного обучения - посмотреть на то, как она обучится и найти проблемы разметки в обучающих данных. Вот такой трикс =)

Единственное, что логично сделать - глянуть ещё несколько изображений, чтобы более уверенно продолжать двигаться с гипотезой, что разметка верна и мы правильно её понимаем.

Но сначала - небольшая вводная в то, что такое изображение в компьютере =)

## Вводная по работе с изображениями

Итак, мы подошли к вопросу, а как же работать с изображениями? Для ответа на этот вопрос нам нужно понять, как изображения представляются в цифровом виде и после уже мы поймём, как их обрабатывать глубокими нейросетями!

В компьютере изображения представлены в виде матриц. Как и привычно, изображения имеют ширину (в пикселях) и высоту (в пикселях). Но матрица, которой представляют цветное изображение, чаще всего имеет три размерности! 

Давайте взглянем на картинку:

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/image_representation.png" width=1200/></p>

Она многое содержит, но главная суть в том, что цветные изображения - это три двумерные матрицы, каждая из которых является отдельным каналом (красный, зелёный или синий). Чёрно-белое изображение - это одна двумерная матрица.

С ЧБ всё достаточно просто, каждый пиксель - это значение в диапазоне $[0; 255]$ (8-бит), которое отражает яркость в этом пикселе.

С цветным всё чуть сложнее, каждый пиксель содержит три значения. Первое - количество красного, второе - зелёного, третье - синего. Смешав цвета с разными интенсивностями, можно получить различные цвета (а именно 16777215 вариантов цветов - 24 бита). То есть по сути каждый пиксель можно представить вот так:

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/image_rgb.png" width=800/></p>

На этом и основывается принцип представления изображений! Это всего лишь двумерная матрица, если изображение чёрно-белое и трёхмерная матрица, у которой последняя размерность равна трём.

> Например, цветное изображение размером 1024х768 - это матрица с размерностью $(768, 1024, 3)$. Ведь сначала идут ряды матрицы (высота), затем колонки (ширина), а затем количество каналов. Такая схема называется HWC ~ Height, Width, Channel.

## Смотрим данные

По данным мы видим, что они расположены в директориях и пока похоже, что изображения чёрно-белые. Давайте взглянем на размер и тип:

In [ ]:
print(f"Shape: {image.shape}")
print(f"Type: {type(image)}")

Из этого вывода видно, что конкретное изображение является массивом numpy (np.ndarray) и имеет размер 28х28 пикселей с тремя каналами. Странно, почему 3, если оно ЧБ?

Причина проста - OpenCV читает изображения по-умолчанию в цветном формате. Даже ЧБ изображение можно представить в цветном формате, только черное будет [0, 0, 0], а белое - [255, 255, 255].

> 🤓 Если быть точным, то OpenCV работает в формате BGR. Это отличный от нами привычного формата RGB для глаза, поэтому если вы начнёте работать с цветными изображениями и увидите, что красный и синий перепутаны - вспомните об этой особенности =)

Если капнуть в документацию:
[Страница датасета](https://www.kaggle.com/scolianni/mnistasjpg) -> [Страница соревнования, на основе которого сделан этот датасет](https://www.kaggle.com/c/digit-recognizer) -> [Инфа об исходном датасете](http://yann.lecun.com/exdb/mnist/index.html)

То можно выяснить, что все картинки должны быть 28х28 и ЧБ.
Это значит, что мы можем изменить вызов функции чтения на следующий:

In [ ]:
image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
image.shape

Как видите, при чтении серого изображения не создаётся третья размерность - её можно считать равной единице. Проверим, что изображение от этого не поменялось:

In [ ]:
plt.figure(figsize=(7,7))
# Ещё добавим подпись для понимания класса разметки
plt.title(str(image_path))
plt.imshow(image, cmap="gray")
plt.show()

### Задание - более широкий взгляд

Отобразите 20 цифр из набора данных в матрице 4х5 с подписями. Каждого класса должно быть в отображении ровно два раза.

> Для отображения plt.imshow() серого изображения нужно убрать одну размерность, так как он принимает (28, 28), а на (28, 28, 1) выдаёт ошибку. Сделать это можно путём img[:, :, 0] или img[..., 0], что равнозначно.

In [ ]:
# TODO - напишите функцию получения N случайных путей до изображений каждого класса
def extract_N_random_class_images(dirpath: Path, N: int) -> typing.List[str]:
    # Пройдитесь по подпапкам директории Path.iterdir() (https://docs.python.org/3/library/pathlib.html#pathlib.Path.iterdir)
    # Вытащите из каждой подпапки N случайных путей до изображений
    # Положите в список для возврата из функции 
    # Обратите внимание на аннотацию, что вернуть надо список строк - не список Path

    raise NotImplementedError()

In [ ]:
# TEST

fpath_list = extract_N_random_class_images(training_set_dirpath, 2)
assert len(fpath_list) == 2*10
assert all([os.path.exists(fpath) for fpath in fpath_list])

In [ ]:
fpath_list = extract_N_random_class_images(training_set_dirpath, 2)

fig, ax = plt.subplots(nrows=4, ncols=5, sharex=True, sharey=True, )
for i, fpath in enumerate(fpath_list):
    _img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)

    class_name = Path(fpath).parts[-2]

    r,c = i//5,i%5
    ax[r,c].imshow(_img, cmap="gray")
    ax[r,c].set_title(class_name)
plt.tight_layout()
plt.show()

Превосходно! Мы увидели, что пока ошибок в разметке не было и всё сходится с гипотезой, что название папки = класс разметки. Шикарно! Давайте перейдём к тому, чтобы, наконец, скормить данные в модель!

## Подготовка изображений

Как уже ранее обсуждалось, PyTorch работает с собственным форматом - тензоры, а это значит, что нам надо подготовить изображения из нынешнего представления (np.ndarray) в формат для PyTorch.

Для этого необходимо сделать несколько действий с готовым изображением:
- Превратить массив numpy в тензор типа `float`;
- Поделить изображение на 255, чтобы привести к диапазону $[0; 1]$;
> Это очень важно для нейросетей - с нормированием данных сеть учится быстрее и меньше попадает на плато в данных. За подробностями к преподавателям =)
- Изменить формат от стандартного HWC к CHW - переставить порядок в матрице.
> Это требование работы со сверточными сетям в PyTorch. Что такое сверточные сети - уже скоро расскажем!

Напишем полный код предобработки изображения:

> Вам понадобятся методы [torch.Tensor.permute()](https://pytorch.org/docs/stable/generated/torch.Tensor.permute.html) и [torch.div()](https://pytorch.org/docs/stable/generated/torch.div.html).

In [ ]:
# TODO - напишите функцию предобработки изображений
def preprocess_image_2_tensor(img: np.ndarray) -> torch.Tensor:
    # img - массив numpy с размерностями HWC
    #   H - высота изображения
    #   W - ширина изображения
    #   C - количество каналов в изображении

    # Метод .permute() - аналог np.transpose(), 
    #   который позволяет поменять порядок размерностей в массиве
    # В этом методе указываются индексы размерностей в том порядке, 
    #   как их надо поставить
    # Изначально было H~0, W~1, C~2
    # Мы хотим поставить C, H, W ~ 2, 0, 1

    raise NotImplementedError

In [ ]:
# TEST
rng = np.random.default_rng(RANDOM_STATE)

_test_arr = rng.random((40, 40, 2))
_test_tnsr = preprocess_image_2_tensor(_test_arr)

np.testing.assert_array_almost_equal(_test_tnsr.shape, (2, 40, 40))
assert torch.is_tensor(_test_tnsr)

Отлично! Данные приведены к требуемому формату! Теперь настало время научиться готовить датасет в PyTorch для загрузки данных.

## Датасет в формате PyTorch

Мы уже сталкивались с разными вспомогательными инструментами PyTorch, а сейчас мы познакомимся с инструментом подгрузки данных. Суть его заключается в том, что мы можем написать реализацию нашего собственного класса с необходимым функционалом подготовки всего одной записи из данных к нужному виду. Далее другие инструменты, используя написанную реализацию, будут загружать целые пачки данных необходимого количества.

Класс зовется [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset).

От слов к делу! Начнем с простой реализации, чтобы понять смысл класса:

In [ ]:
class MySimpleDataset(torch.utils.data.Dataset):
    # Аргумент count здесь для примера, 
    #   можно делать любые аргументы конструктору
    def __init__(self, count):
        self.numbers = np.arange(count)
    
    # Метод __len__() позволяет применять функцию len() к объекту
    #   такого класса
    def __len__(self):
        return len(self.numbers)

    # Метод __getitem__() позволяет обращаться к объекту
    #   такого класса по индексу
    def __getitem__(self, index):
        return torch.tensor([self.numbers[index]])

Вот так выглядит минималистичный пример класса датасета. В нём реализованы все необходимые методы. По сути необходима реализация метода `__len__()` для получения размера датасета и метода `__getitem()__` для получения данных из датасета по индексу. Посмотрим, как обращаться с этим датасетом:

In [ ]:
my_dataset = MySimpleDataset(10)

# Вот здесь будет вызван метод __len__() класса
# Так можно получить размер датасета
print(f"Length: {len(my_dataset)}")

# Вот здесь будет вызван метод __getitem__() класса
# Таким образом можно получить данные из датасета
print(f"Item with index 3: {my_dataset[3]}")
print(f"Item shape with index 3: {my_dataset[3].shape}")

Отлично, как видите, созданный класс позволяет получать данные из какого-то источника универсальным способом. Например, мы можем написать такой класс, который будет сам читать директорию и выдавать файл по индексу. 

Круто? Круто!

> За доками методов сюда: [object.__getitem__](https://docs.python.org/3/reference/datamodel.html#object.__getitem__), [object.__len__](https://docs.python.org/3/reference/datamodel.html#object.__len__) 

Теперь давайте посмотрим на инструмент [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), который позволяет на основе датасета сделать получение батчей данных, перемешивание и много чего другого.

In [ ]:
# Создадим объект загрузчика
my_dataloader = torch.utils.data.DataLoader(
    dataset=my_dataset, # Передаём наш датасет в загрузчик 
    batch_size=4,       # Задаём размер батча для подгрузки
    shuffle=True        # Сообщаем о том, что нам нужны перемешанные данные
)

# Размер загрузчика представляется уже в количестве батчей
#   4+4+2 (остаток)
print(f"DataLoader length: {len(my_dataloader)}")

# Посмотрим, как загрузчик выдаёт данные
# Так как он не имеет индексации, то один из простых способов - 
#   пройти по загрузчику циклом
for batch in my_dataloader:
    print("----")
    print(batch)
    print(f"Shape: {batch.shape}")

Таким образом, мы написали класс для подготовки тензора по одной записи в данных, а загрузчик уже объединяет их в батчи. Причём, наш датасет выдаёт массив размером $(1)$, а загрузчик объединяет такие данные и добавляет размерность $(4, 1)$. То есть из формата $K$ делается формат $NK$, где $K$ - размер данных, выдаваемых датасетом, $N$ - размер батча или остатка от него.

Это очень удобно, так как мы пишем реализацию подготовки всего для одного изображения, а далее инструменты объединяют их в батчи и при необходимости перемешивают между собой. Круто!

Давайте теперь вернемся к нашей задаче распознавания цифр и напишем свой датасет с блекджеком и подготовкой изображений!

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/bender.png" width=600/></p>

### Задание - пишем датасет изображений

В конструктор датасета будет передан путь до директории с тренировочными изображениями. 

**Задача** - написать реализацию датасета, чтобы он мог выдавать изображения в подготовленном формате!

In [ ]:
# TODO
class DigitsDataset(torch.utils.data.Dataset):
    def __init__(self, dirpath: Path):
        # Прямо в конструкторе сформируем список путей до файлов
        raise NotImplementedError

    def __len__(self) -> int:
        # Верните количество изображений в наборе
        raise NotImplementedError

    def __getitem__(self, index: int) -> typing.Tuple[torch.Tensor, int]:
        # Получите изображение и разметку по индексу
        
        # Подготовьте изображение для работы в PyTorch
        #   float, [0; 1], CHW

        # Обратите внимание, что OpenCV читает в формате HW серые изображения
        #   Нужно добавить в конец ещё одну размерность

        # Верните tuple, состоящий из двух элементов:
        #   - тензор изображения
        #   - тензор значения разметки (истинное значений цифры на изображении - вытащите из пути до файла и преобразуйте в int, а затем в тензор)
        raise NotImplementedError

        return img_tnsr, label

In [ ]:
# TEST
digits_dataset = DigitsDataset(training_set_dirpath)

img_tnsr, label = digits_dataset[100]

assert torch.is_tensor(img_tnsr)
np.testing.assert_array_almost_equal(img_tnsr.shape, torch.Size((1, 28, 28)))
np.testing.assert_almost_equal(img_tnsr.max(), 1)
np.testing.assert_almost_equal(img_tnsr.min(), 0)

# Вернём обратно из тензора в изображение, чтобы отобразить
def tensor_2_image(tnsr: torch.Tensor) -> np.ndarray:
    img = tnsr.permute(1, 2, 0).numpy()
    img = (img*255).astype(np.uint8)
    return img

_img = tensor_2_image(img_tnsr)

plt.imshow(_img[..., 0], cmap="gray")
plt.title(f"Label: {label}")
plt.show()

Если в результате теста вы видите цифру и она сходится с надписью над ней - всё подготовлено верно! Теперь можно создать загрузчик и убедиться, что данные выдаются в виде батчей:

In [ ]:
digits_dataloader = torch.utils.data.DataLoader(
    dataset=digits_dataset,
    batch_size=4,
    shuffle=False
)

for imgs_batch, labels_batch in digits_dataloader:
    print(f"Shape: {imgs_batch.shape}")
    print(f"Labels batch: {labels_batch}")

    # Просто проверка, не нужно проходить по всему!
    break

# Ещё для проверки можно преобразовать загрузчик в итератор 
#   и вызвать функцию next()
print("---")
imgs_batch, labels_batch = next(iter(digits_dataloader))
print(f"Shape: {imgs_batch.shape}")
print(f"Labels batch: {labels_batch}")

Замечательно! На этом подготовка изображений завершена! Мы бегло выполнили анализ данных, преобразовали в нужный формат и написали класс датасета, который позволяет получать данные в формате, требуемом для обучения! Самое время перейти к разработке модели свёрточной сети!

## Из чего состоит свёрточная нейросеть

Модели свёрточных сетей считаются глубокими сетями и относятся к области глубокого обучения. Почему? Причина в их сложности. Это не просто группы нейронов, которые соединены между собой. Давайте разбираться, что к чему!

Основная идея заключается в том, что ранее мы работали с данными, признаки которых представляли собой конкретные показатели и по ним принималось то или иное решение моделью. В случае изображений признаками являются интенсивности пикселей, но по отдельности они несут крайне бесполезную информацию. Совокупность пикселей хотя бы в небольшой области может дать уже намного бОльшую информацию, именно поэтому свёрточные сети показали великолепные результаты, так как сама операция свёртки (из названия) производит обработку изображения по областям.

Глубоким обучением данная область считается, потому что сеть сама ищет и генерирует признаки, исходя из данных. В этом плане с одной стороны проще, так как не требуется большой работы с предобработкой признаков, с другой - многое теперь зависит от модели, количества данных (а для глубокого обучения их нужно больше), и от подготовки этих данных путём анализа шума и фильрации выбросов.

Схематично свёрточную сеть можно представить следующим образом:

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/convolution_nn.png" width=600/></p>

Видно, что она состоит из двух частей:
- Feature learning - генерация признаков, на этом этапе модель состоит только из свёрточных слоёв, на которых генерируются признаки изображения;
- Classification - классификация как частный случай, скорее часть принятия решения, ставится обычная полносвязная нейросеть, которая на основе сгенерированных из изображения признаков решает конечную задачу.

> ⚠️ Ещё обратите внимание, в свёрточных сетях тоже есть слои!

Видно, что для примера представлена задача классификации, на вход (слева) поступает изображение, а задача модели - классифицировать эту картинку. Вообще, мы такую задачу и планируем сейчас решать, только наши предсказываемые классы - цифры от 0 до 9, то есть 10 классов!

> ⚠️ Давайте для закрепления проведём параллель, значения интерсивностей пикселей - это и есть базовые фичи в данных. Просто, как, например в случае леса, бесполезно будет пытаться принимать решение на основе значения интенсивности в пикселе на 20-й строке 15-й колонки. Свёрточные сети генерирую новые признакми на основе групп пикселей и их интенсивностей. По сути, изображение - это уже численная матрица в компьютере, поэтому задача подготовки данных для подачи на вход модели ваыполнена. Просто, раньше мы работали с одномерными данными (векторы), а теперь сами данные стали сложнее =)

Давайте теперь пройдемся по основным составляющим модели свёрточной сети, причём мы опустим последний слой, так как ничего нового мы о нём не узнаем.

### Операция свертки

> Спасибо [статье](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) за картинки!

Операция свёртки известна из многих областей, но в обработке изображения она выглядит следующим образом:

![Текст ссылки](https://miro.medium.com/max/535/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif)

> Если у вас не отображается гифка - скопируйте ссылку в браузер

Так что здесь проиходит? На изображении есть ядро размером $(3, 3)$, которое проходит по всему изображению слева направо на самом верху, затем опускается на один пиксель вниз и далее слева направо. Ядро имеет свои значения, пиксели на изображении тоже. Операция свёртки заключается в том, что ядро, попадая на область изображения вычисляет сумму произведений поэлементных значений ядра и элементов области. Далее эта сумма записывается в результат. Давайте разберём верхний правый угол. Ядро имеет вид:
$$
\begin{bmatrix}
0 && 1 && 2 \\
2 && 2 && 0 \\
0 && 1 && 2
\end{bmatrix}
$$

Верхняя правая область имеет вид:
$$
\begin{bmatrix}
2 && 1 && 0 \\
1 && 3 && 1 \\
2 && 2 && 3
\end{bmatrix}
$$

Таким образом, сумма элементов будет равна:
$$
\sum = 0*2 + 1*1 + 2*0 + \\
2*1 + 2*3 + 0*1 + \\
0*2 + 1*2 + 2*3 = 1 + 8 + 8 = 17
$$

Результат 17 так и записан в верхний правый элемент результата! Отлично, всё сошлось!

Теперь, в чём тут идея? Да именно в том, что сеть анализирует ядро области изображения и "преобразует" информацию, тем самым создавая новые "внутренние" представления для генерации признаков.

> ⚠️ Но как слой свёртки учится? В нейроне были веса входа, а здесь непонятно. Легко запомнить - изображение меняться не может, так что это фиксированная величина, поэтому ядро слоя свёртки является обучаемым. Точнее, значения в ядре являются параметрами слоя и в ходе обучения настраиваются.

> ⚠️ Размер ядра - гиперпараметр операции свёртки (слоя сети).

Теперь надо затронуть ещё пару аспектов свёртки, так как эта операция является основополагающей в свёрточных сетях!

#### Паддинг (отступы)

Самые наблюдательные обратят внимание, что все писели изображения кроме крайних могут попасть в центр ядра и активно участвуют в операции свертки. Крайние пиксели участвую в операции свёртки реже, чем остальные пиксели, что сказывается на получении информации с краёв изображения - плохо, ведь крайние пиксели могут нести полезную информацию. А что, если размер ядра $(11, 11)$? Тогда не один ряд крайних не участвует, а целых 5 рядов!

> Вам понятно, как 5 было получено? Ядро специально берётся нечётного размера, чтобы был "центральный пиксель". Таким образом в стандартном виде 5 пикселей с краю никогда не станут центральными (размер ядра/2 с округлением вниз).

Так вот **паддинг** (отступ), как он выглядит:

![Текст ссылки](https://miro.medium.com/max/395/1*1okwhewf5KCtIPaFib4XaA.gif)

Идея заключается в добавлении отступов с краёв, чтобы все пиксели в изображении равномерно участвовали в формировании полезной информации! Размер отступов определяется размером ядра в слое свертки. По сути, паддинг несёт два назначения:
- Позволяет получать информацию со всего изображения без исключения краёв;
- Позволяет формировать результат такой же по размеру, как и вход.

> Свёрточная сеть составлена из операций свёртки, поэтому входом не всегда являются изображения, а чаще всего входом является выход предыдущего слоя свертки.

> Паддинг чаще всего выбирается исходя из размера ядра, но может также считаться гиперпараметром слоя.

#### Stride (шаг свёртки)

Вторая особенность для обсуждения - шаг, с которым свёртка проходит по изображению. В оригинале мы рассмотрели шаг, равный единице, когда свертка проходит пиксель за пикселем. Можно в качестве страйда выбрать шаг равный двум пикселям, тогда это будет выглядеть следующим образом:

![Текст ссылки](https://miro.medium.com/max/294/1*BMngs93_rm2_BpJFH2mS0Q.gif)

Как видно, между шагами появился пропуск, а также результат стал заметно меньше! Это как раз характеризует страйд. При сравнении шага равного единице и двум, можно отметить, что вычисление производится быстрее, а также результат содержит меньше информации. В некоторых случаях это хорошо, тем более мы скоро рассмотрим слой отвечающий за прореживание информации, но выбор шага определяет размер выхода (во сколько раз он будет уменьшен). Например, размер изображения 400х400, страйд выбирается равный двум, так что результат будет иметь размер 200x200.

> Страйд является гиперпараметром слоя свертки.

#### Dilation (разреженность)

Этот аспект свёртки является достаточно специфичным и связан с таким понятием как **поле восприятия (receptive field)**. Данное понятие в данный момент рассматривать не будет, лишь отметим суть показателя разреженности:

![Текст ссылки](https://miro.medium.com/max/395/0*3cTXIemm0k3Sbask.gif)

Стандртная свёртка имеет dilation rate равный единице. Анимация показывает суть работы при показателе равном двум. То есть ядро растягивается сильнее и получает пропуски в себе же. Информация таким образом собирается шире (это если кратко про поле восприятия и про то, что так оно расширяется), так как часто соседние пиксели не несут столько полезной информации, сколько несёт более широкая зона, но с пропусками пикселей через один.

> Dilation rate также является гиперпараметром


### Свётка в PyTorch

В качестве небольшой практики попробуем операцию свёртки отдельно ото всех остальных, чтобы немного разобраться в ней. Как всегда создаём модуль, отвечающий за эту операцию:

In [ ]:
conv_mod = nn.Conv2d(
    in_channels=1,  # Количество входных каналов = 
                    #   количество выходных каналов предыдущей свёртки 
                    #   (или каналов входного изображения, если это первый слой)
    out_channels=1, # Количество выходных каналов = количество ядер
    kernel_size=3,  # Размер ядра
    stride=1,       # Страйд по-умолчанию,
    padding=0       # Нулевой отступ
)

Тут важно отметить, что количество выходных каналов определяет количество различных ядер внутри свёртки. Так, например, даже если входной канал один, то выходных может быть 16, тогда внутри свёртки будут обучаться 16 разных ядер, которые применяются каждое ко входу и формируют 16 разных выходов.

Теперь давайте проверим атрибуты модуля и попробуем применить свёртку к случайному изображению:

In [ ]:
print(conv_mod.weight)
print(f"Weights shape: {conv_mod.weight.shape}")

Размер весов соответвует формату: `out_channels, in_channels, kernel_size, kernel_size`.

Давайте посмотрим на размер весов с другим количеством каналов:

In [ ]:
_shape = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3).weight.shape
print(f"Weights shape: {_shape}")

Окей, разобрались с тем, как в свертке расположены веса (параметры обучения), а теперь давайте капнём в то, как применять свертку:

In [ ]:
# Делаем входной тензор с форматом BCHW ~ 1,1,5,5
X_rnd_img = torch.randint(low=0, high=2, size=(1, 1, 5, 5)).float()
print(X_rnd_img)

In [ ]:
# Заполним веса единицами и выполним операцию
conv_mod.weight.data.fill_(1)
# В свертке есть bias, его мы обнулим
conv_mod.bias.data.fill_(0)
print(conv_mod.weight)
print(conv_mod.bias)

result = conv_mod(X_rnd_img)
print(result)

По результатам можно увидеть вычисление результата, соответствующее ожиданиям расчёта. Мы не обсуждали bias, но тут всё просто - это просто число для каждого ядра, которое прибавляется ко всему ядру. По аналогии с bias в нейроне.

Таким образом, по аналогии с полносвязной сетью, где мы создавали линейный слой нейронов, здесь мы создаём линейный слой ядер. К чему эта аналогия? Для свертки тоже нужна нелинейность, рассмотрим одну из известных для сверточных сетей нелинейность - ReLU!

### Нелинейность для свертки

Как мы уже изучали, нейронная сеть без нелинейности - GPU на ветер. Поэтому глянем на штуку под названием Rectified Linear Unit (ReLU). Эта нелинейность выглядит очень просто, тем не менее является полноценной нелинейностью и показывает отличные результаты, хотя и на сегодняшний день есть огромное количество аналогов!

Выглядит она следующим образом:

In [ ]:
x = np.linspace(-10, 10, 300)
y = np.maximum(x, 0)

plt.figure(figsize=(7,5))
plt.plot(x, y)
plt.title("ReLU")
plt.grid()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Даже не будем писать формулу, просто максимум между значением и нулем. Наверное, было бы правильно рассмотреть её производную, всякие свойства, но мы просто попробуем применить её в PyTorch:

In [ ]:
x_tnsr = torch.tensor([-2, -1, 0, 1, 2]).float()
relu_op = nn.ReLU()

relu_op(x_tnsr)

Вот и всё, нелинейность очень простая, тем не менее её важно размещать в свёрточной нейронной сети, чтобы добавить ей свойство нелинейности.

### MaxPooling

Пулинг в сверточной нейронной сети является этапом уменьшения размерности вместе со сжатием информации. Представить его можно следующим образом:

![Текст ссылки](https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png)

По сути, это ядро, которое не имеет значений, а просто берёт максимум из области, на которое попадает и шагает со страйдом, равным размеру ядра. На изображении показан MaxPooling с ядром размером $(2, 2)$.

Данная операция является очень полезной с точки зрения преобразования данных из изображения в вектор признаков, на основе которого полносвязная сеть принимает решения.

В свёрточных сетях происходит постепенный рост размерности по каналам (C) и постепенное уменьшение размерности по сторонам (HW). Таким образом, картинка постепенно уменьшается и в конце концов превращается в одномерный вектор, при этом этот вектор уже представляет собой не пространственную информацию, а информацию в виде вектора признаков, удобном для принятия решения по конкретной задаче.

> Размер ядра (размер пулинга) является гиперпараметром слоя

Думаю, стоит сразу перейти к практике с модулем в PyTorch:

In [ ]:
pool_mod = nn.MaxPool2d(
    kernel_size=2,  # Размер ядра пулинга
    # stride=2      # Можно задать страйд, но он по-умолчанию равен размеру ядра
)

x_rnd_img = X_rnd_img = torch.randint(low=0, high=20, size=(1, 1, 4, 4)).float()
print(x_rnd_img)

In [ ]:
result = pool_mod(x_rnd_img)
print(result)

Вот так мы быстренько рассмотрели три основных составляющий свёрточной нейронной сети. По сути из них только операция свёртки является обучающей, остальные две добавляют сети свойств, благодаря которым она может уверенно обучаться и давать предсказания с высокой точностью.

> Рассмотренные компоненты свёрточной сети являются минимумом из существующих сегодня. Такие вещи как регуляризация с помощью [BatchNorm](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html), восстановление изображения с помощью [Deconvolution ~ ConvolutionTransposed](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html) и другие, опущены в данной практике, но тем не менее рекомендованы к изучению в дальнейшем! 

## Составим нашу первую сверточную сеть!

Сеть мы составим простую, будет всего два слоя, каждый из которых будет иметь нелинейность и слой пулинга. 

После этих слоёв нам нужно преобразовать тензор с тремя размерностями CHW в тензор с одной размерностью за счёт flattening (reshape).

In [ ]:
class MyFirstConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(RANDOM_STATE)
        # Делаем первый блок свертки
        # Воспользуемся классом nn.Sequential, чтобы сделать
        #   один модуль из нескольких
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # После пулинга размер из 28х28 преобразуется в 14х14

        # Делаем второй блок свертки
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # После пулинга размер из 14х14 преобразуется в 7х7
        
        # fc ~ fully connected block
        self.fc = nn.Sequential(
            # Первый слой для входа - это выпрямленный вектор
            # Размер выбирается исходя из предыдущего слоя
            # H*W*C, HxW = 7x7, C = кол-во каналов на пред. слое
            nn.Linear(7*7*32, 500),
            nn.Sigmoid(),
            # Выход последнего слоя по количеству предсказываемых классов
            # Размерность выхода - NK (K=10)
            nn.Linear(500, 10)
            # На этом уровне предсказаны Logits - сырые степени уверенности
            # Далее мы разберём, что функция потерь это и ожидает, поэтому
            #   на этом блок закончен
        )
    
    def forward(self, x):
        # Делаем предсказания блоков
        out = self.layer1(x)
        out = self.layer2(out)
        # Делаем flattening результатов
        # Из NCHW делаем NV
        # N - размер батча, V = C*H*W
        out = out.reshape(out.size(0), -1)
        # Предсказания последним блоком
        out = self.fc(out)
        return out

Теперь, после написания класса важно сделать проверку того, что связи настроены верно, а если нет, то можно будет быстро отладить, пока не написан большой код.

In [ ]:
# TEST - просто проверить, что работает =)
model = MyFirstConvNet()

# Сделаем случайный батч для проверки
# 2 записи в батче, CHW - соответсвует нашим картинкам
X_rndm_batch = torch.randn((2, 1, 28, 28), dtype=torch.float)
y_pred = model(X_rndm_batch)

assert np.all(y_pred.shape == torch.Size((2, 10)))

## Функция потерь

Отлично! Наша модель написана и протестирована! Самое время разобраться с тем, как нам оценивать работу модели - копаем в [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

В этом примере рассмотрим абстрактную ситуацию: 

**задача** - мультиклассовая классификация по четырём классам. Модель выдаёт сырые степени уверенности (logits). Глядим, как отрабатывает функция потерь в разных ситуациях:

In [ ]:
# Создаём модуль
loss_op = nn.CrossEntropyLoss()

# Для начала сделаем идеальный случай
# Тензор истинных значений должен быть одномерным c размером N и в нём указываются
#   индексы классов
y_true = torch.tensor([1])
# Тензор предсказаний должен быть NK (K - кол-во классов) 
#   с сырыми степенями уверенности
print(f"Ground truth shape: {y_true.shape}")

# Вариант 1, степень уверенности у индекса 3 выше всех, плохое предсказание
y_pred_1 = torch.tensor([[0, 0, 0, 2]]).float()
print(f"Prediction 1 shape: {y_pred_1.shape}")
# Вариант 2, индекс 0 выше всех, но также имеются уверенности у 2 и 3,
#   хуже, чем вариант 1, так как степень уверенности выше у 0,
#   чем в первом варианте у 3
y_pred_2 = torch.tensor([[10, 0, 3, 1]]).float()
print(f"Prediction 2 shape: {y_pred_2.shape}")
# Вариант 3, наибольший индекс 1, также имеются уверенности у остальных,
#   то что надо
y_pred_3 = torch.tensor([[10, 100, 3, 1]]).float()
print(f"Prediction 3 shape: {y_pred_3.shape}")

# Попробуем разные вектора предсказаний и сравним их
print(f"Loss 1: {loss_op(y_pred_1, y_true)}")
print(f"Loss 2: {loss_op(y_pred_2, y_true)}")
print(f"Loss 3: {loss_op(y_pred_3, y_true)}")

Вот так мы эмпирически разобрали, как функция потерь ведёт себя. В документации есть большая и страшная формула, но главная её суть в следующем: мы передаем желаемый индекс класса и сырые степени уверенности, которые выдаёт модель по всем классам. Дальше, если степень уверенности по желаемому индексу выше остальных - значит модель правильно работает и всё хорошо. В ином случае - надо модель доучить и поэтому выдаётся Loss значение выше нуля.

> ✨ Если интересно, вы можете вынести на обсуждение специфику функции потерь и обсудить, почему это работает так. С точки зрения применения помните, какие форматы нужно подавать на вход и всё будет ок.

> ⚠️ Помимо этой функции потерь в мультиклассовой классификации применяется куча других, например, FocalLoss, NLLoss. У каждой есть свои плюсы, но это оставляем на самостоятельный разбор =)

По формуле на сайте можно увидеть, что реализация PyTorch включает в себя Softmax, так что на вход предсказания ожидаются сырые степени уверенности (logits).

Сейчас мы убедились, что функция потерь работает логично и мы освоили способ, как с ней работать. Настало время написать цикл обучения!

## Пора учить!

Как известно, лучшие практики предполагают разделение данных на выборки, чтобы можно было на обучающей проводить обучение, на валидационной искать наилучшие гиперпараметры, а на тестовой производить конечную оценку!

У нас есть директория в файлами - нам нужно разделить её на три директории с файлами!

Давайте разделим наш датасет на выборки обучения, валидации и теста. Тест (test) должен быть 20% от всего набора, тогда останется 80% на обучение (train_val). Затем валидация (val) отделяется от полученной выборки обучения (train_val) на 25%, а оставшееся (train) используется для обучения. Так мы получим 60-20-20, если смотреть по общему количеству данных. И не забывайте про стратификацию!

In [ ]:
from sklearn.model_selection import train_test_split
import shutil

def split_data(dirpath: Path, new_dir_name: str) -> typing.Tuple[Path, Path, Path]:
    labels = []
    paths = []
    
    for class_path in dirpath.iterdir():
        class_label = class_path.parts[-1]
        for path in class_path.iterdir():
            paths.append(path)
            labels.append(class_label)

    all_idxs = range(len(paths))

    train_idxs, test_idxs = train_test_split(
        all_idxs, test_size=0.2, random_state=RANDOM_STATE, stratify=labels
    )

    train_val_idxs = [all_idxs[i] for i in train_idxs]
    train_val_labels = [labels[i] for i in train_idxs]

    train_idxs, val_idxs = train_test_split(
        train_val_idxs, test_size=0.25, random_state=RANDOM_STATE,
        stratify=train_val_labels
    )

    print(f"Train size: {len(train_idxs)}")
    print(f"Val size: {len(val_idxs)}")
    print(f"Test size: {len(test_idxs)}")

    # Индексы разделили, теперь можно скопировать файлы в новые директории

    new_dataset = Path(new_dir_name)

    train_dataset = new_dataset / Path("train")
    valid_dataset = new_dataset / Path("valid")
    test_dataset = new_dataset / Path("test")

    dataset_idx_pairs = [
        (train_dataset, train_idxs),
        (valid_dataset, val_idxs),
        (test_dataset, test_idxs),
    ]

    for target_dataset_dirpath, idxs in dataset_idx_pairs:
        target_dataset_dirpath.mkdir(parents=True, exist_ok=True)

        for idx in idxs:
            path = paths[idx]
            label = labels[idx]

            class_path = target_dataset_dirpath / Path(label)
            new_path = class_path / path.name

            class_path.mkdir(exist_ok=True)
            shutil.copyfile(path, new_path)

    return train_dataset, valid_dataset, test_dataset


In [ ]:
train_dirpath, valid_dirpath, test_dirpath = split_data(training_set_dirpath, new_dir_name="splitted")

Ух, не быстро, но машинка осилила =)

Замечательно! Теперь мы подготовили данные и можем подготовить загрузчики данных, чтобы затем использовать их в обучении!

In [ ]:
def prepare_loaders(config):
    # Обратите внимание тут мы передаем конфиг, в котором прописаны параметры обучения
    train_dirpath = config["train_dirpath"]
    valid_dirpath = config["valid_dirpath"]
    test_dirpath = config["test_dirpath"]
    batch_size = config["batch_size"]

    # Готовим загрузчики для каждой выборки
    train_loader = torch.utils.data.DataLoader(
        dataset=DigitsDataset(train_dirpath),
        batch_size=batch_size,
        shuffle=True
    )

    val_loader = torch.utils.data.DataLoader(
        dataset=DigitsDataset(valid_dirpath),
        batch_size=batch_size,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        dataset=DigitsDataset(test_dirpath),
        batch_size=batch_size,
        shuffle=True
    )

    return train_loader, val_loader, test_loader

Загрузчики готовы, теперь переходим к разбору основных циклов обучения и валидации! Для начала напишем функцию обучения одной эпохи:

In [ ]:
def train_epoch(model, loader, loss, optim):
    history = {
        "loss": []
    }

    # Таким образом переводим модель в режим обучения
    # В этом режиме вычисляются градиенты, нужные для обучения
    model.train()
    for imgs_batch, labels_batch in loader:
        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history["loss"].append(loss.item())

        optim.zero_grad()
        loss.backward()
        optim.step()
    
    return history

In [ ]:
# TEST

# Зададим параметры обучения в единой структуре
config = {
    # Коэффициент обучения, определяет величину изменения весов
    "lr": 0.01,
    # Количество эпох
    "epochs": 5,
    # Размер батча - обычно определяется количество досутпной памяти на компьютере
    "batch_size": 32,

    "train_dirpath": train_dirpath,
    "valid_dirpath": valid_dirpath,
    "test_dirpath": test_dirpath,
}

train_loader, val_loader, test_loader = prepare_loaders(config)

model = MyFirstConvNet()
loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=config["lr"])

epoch_train_history = train_epoch(model, train_loader, loss_op, optim)

assert len(epoch_train_history["loss"]) == len(train_loader)

Теперь перейдем к этапу валидации:

In [ ]:
def valid_epoch(model, loader, loss):
    history = {
        "loss": []
    }

    # Таким образом переводим модель в режим исполнения (inference)
    # В этом режиме отключены градиенты, он быстрее, 
    #   но в нём нельзя обучать модель
    model.eval()
    for imgs_batch, labels_batch in loader:
        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history["loss"].append(loss.item())
    
    return history

In [ ]:
# TEST
epoch_val_history = valid_epoch(model, val_loader, loss_op)

assert len(epoch_val_history["loss"]) == len(val_loader)

Теперь мы можем объединить написанный код в полный цикл обучения:

In [ ]:
config = {
    "lr": 0.01,
    # Для проверки - 2 эпохи
    "epochs": 2,
    "batch_size": 32,

    "train_dirpath": train_dirpath,
    "valid_dirpath": valid_dirpath,
    "test_dirpath": test_dirpath,
}

train_loader, val_loader, test_loader = prepare_loaders(config)

model = MyFirstConvNet()
loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=config["lr"])

history = {
    "train_loss": [],
    "val_loss": []
}

for epoch in range(config["epochs"]):
    # Сначала выполняем цикл обучения
    epoch_train_history = train_epoch(model, train_loader, loss_op, optim)
    # Затем исполняем цикл валидации
    epoch_val_history = valid_epoch(model, val_loader, loss_op)

    # В историю обучения попадает среднее по эпохе значение
    epoch_train_loss = np.mean(epoch_train_history["loss"])
    epoch_val_loss = np.mean(epoch_val_history["loss"])
    history["train_loss"].append(epoch_train_loss)
    history["val_loss"].append(epoch_val_loss)

    print(f"Epoch {epoch}:")
    print(f"  Train loss: {epoch_train_loss}")
    print(f"  Valid loss: {epoch_val_loss}")

print(history)

Заметим два важных факта:
- Код работает и не выдает ошибок; Всегда лучше потратить дополнительно время и лишний раз проверить входы и выходы, на предмет ошибок и расхождений.
- Функция потерь уменьшается как на обучении, так и на валидации - это хороший знак!

На этот момент у нас написан код для загрузки данных, обучения, валидации и код модели! Вроде можно учить модель и далее оценивать её работу, но мы ещё не использовали всей мощи обучения. Пора посмотреть, как нашу модель учить с использованием GPU!


## Применение аппаратного ускорения для обучения и исполнения

PyTorch отличается очень удобным интерфейсом по реализации вычислений на конкретных ускорителях. 

> Если вы работаете в Google Colab, то с этого момента рекомендуется проверить, что у вас включен GPU ускоритель: Среда выполнения -> Сменить среду выполнения -> Выберите аппаратный ускоритель GPU. При смене ускорителя ваша среда перезапускается, поэтому удобно выполнить ячейки до этой в вкладке "Среда выполнения" или комбинацией Ctrl+F8. Помните, что при перезапуске среды ваш файл `kaggle.json` удалится, так что перед перезапуском надо загрузить его снова.

> Если вы работаете на своей машине и у вас нет GPU, то не беда. Мы будем писать код, который подстраивается под использование на том или ином вычислителе.

> Если вам не нужно в ноутбуке использовать GPU, то рекомендуется отключать ускоритель, чтобы не занимать важные ресурсы. Так код переходит на исполнение на CPU.

Для проверки доступности GPU PyTorch имеет удобный интерфейс, проверим его:

In [ ]:
# Данная булева проверка вернет True, 
#   если имеется CUDA устройство (GPU от NVidia)
torch.cuda.is_available()

Отлично, если проверка пройдена, то вы можете использовать CUDA устройство. Обычно для применения делается следующий выбор устройства из доступных:

> ✨ Если у вас не найдена CUDA или попросту нет видеокарты, то далее всё также должно работать, но сильно дольше, чем на GPU. Вы можете выполнить этот нойтбук на Google Colab, чтобы убедиться в ускорении работы. Не забудьте выбрать там устройство для исполнения - GPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Таким образом, мы получили объект устройства, на котором планируется исполнение. Теперь, чтобы выполнять вычисления на GPU (в том числе обучение), нужно, чтобы на вход модулей поступали тензоры, которые размещены на устройстве для вычисления. Для этого есть метод `torch.Tensor.to()`, в который можно передать объект устройства, на котором будем производить вычисления. Давайте создадим тензор и перенесём его на устройство:

> Доки метода [torch.Tensor.to()](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html). А вообще, можно сразу создавать тензоры на девайсе, указывая правильный аргумент `device` в конструкторе [torch.tensor()](https://pytorch.org/docs/stable/generated/torch.tensor.html).

In [ ]:
smpl_tensor = torch.tensor([1, 2, 3]).float()
print(smpl_tensor) 

# При переносе тензора на устройство важно сделать присвоение,
#   так как тензор не сам переносится, а метод `.to()` возвращает
#   в качестве результата тензон на устройстве
smpl_tensor = smpl_tensor.to(device)
print(smpl_tensor)

По сути, для обучения на GPU требуется:
- Перенести модель на устройство (у `nn.Module` тоже есть метод `.to()`);
- Перенести входной тензор батча на устройство;
- Перенести тензор разметки на устройство;
- Произвести вычисление предсказания модели;
- Вычислить Loss (заметьте, всё ещё на устройстве);
- (*) Если предсказания планируется использовать в numpy (или по-другому, но вне GPU), то надо вернуть тензор на CPU методом [torch.Tensor.cpu()](https://pytorch.org/docs/stable/generated/torch.Tensor.cpu.html).

Давайте напишем новые функции обучения эпох с учетом устройства для выполнения!

In [ ]:
def train_epoch(model, loader, loss, optim, device):
    history = {
        "loss": []
    }

    # Таким образом переводим модель в режим обучения
    # В этом режиме вычисляются градиенты, нужные для обучения
    model.train()
    for imgs_batch, labels_batch in loader:
        imgs_batch = imgs_batch.to(device)
        labels_batch = labels_batch.to(device)

        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history["loss"].append(loss.item())

        optim.zero_grad()
        loss.backward()
        optim.step()
    
    return history

def valid_epoch(model, loader, loss, device):
    history = {
        "loss": []
    }

    # Таким образом переводим модель в режим исполнения (inference)
    # В этом режиме отключены градиенты, он быстрее, 
    #   но в нём нельзя обучать модель
    model.eval()
    for imgs_batch, labels_batch in loader:
        imgs_batch = imgs_batch.to(device)
        labels_batch = labels_batch.to(device)
        
        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history["loss"].append(loss.item())
    
    return history

In [ ]:
# TEST
config = {
    "lr": 0.01,
    # Для проверки - 2 эпохи
    "epochs": 2,
    "batch_size": 32,

    "train_dirpath": train_dirpath,
    "valid_dirpath": valid_dirpath,
    "test_dirpath": test_dirpath,
}

train_loader, val_loader, test_loader = prepare_loaders(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyFirstConvNet()
model.to(device)

loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=config["lr"])

history = {
    "train_loss": [],
    "val_loss": []
}

for epoch in range(config["epochs"]):
    # Сначала выполняем цикл обучения
    epoch_train_history = train_epoch(model, train_loader, loss_op, optim, device)
    # Затем исполняем цикл валидации
    epoch_val_history = valid_epoch(model, val_loader, loss_op, device)

    # В историю обучения попадает среднее по эпохе значение
    epoch_train_loss = np.mean(epoch_train_history["loss"])
    epoch_val_loss = np.mean(epoch_val_history["loss"])
    history["train_loss"].append(epoch_train_loss)
    history["val_loss"].append(epoch_val_loss)

    print(f"Epoch {epoch}:")
    print(f"  Train loss: {epoch_train_loss}")
    print(f"  Valid loss: {epoch_val_loss}")

print(history)

Теперь после добавления выбора устройства (точнее выбор GPU, если доступен, иначе CPU) мы можем ускорить наше обучение с помощью GPU или как минимум работать на всегда доступном CPU!

## А как оценивать то?

Во время обучения значение функции потерь на валидации - дело, конечно, полезное, но не совсем понятное. Эти значения нельзя никак интерпретировать. Поэтому, дополнительно применяется оценка по определенным метрикам, которые позволят понять хотя бы банально accuracy нашей модели на конкретной эпохе. Давайте по аналогии с функциями `train_epoch()` или `valid_epoch()` напишем функцию `evaluate_loader()`, который будет возвращать словарь с желаемыми метриками. Для начала это будет accuracy.

Мы сначала напишем функцию оценки accuracy по тензорам предсказания и разметки:

In [ ]:
# TODO
def evaluate_batch_accuracy(y_pred, y_true):
    """
    Оценка точности предсказания (accuracy)

    y_pred:
        батч сырых степеней уверенности, размер (N, K)
    y_true:
        вектор истинных значений, размер (N)
    """
    return None

In [ ]:
# TEST

y_true = torch.tensor([1, 2, 1])
y_pred = torch.tensor([
    [0, 0, 0, 2],
    [10, 10, 12, 0],
    [1, 10, -1, 2]
]).float()

np.testing.assert_almost_equal(evaluate_batch_accuracy(y_pred, y_true), 0.66, decimal=2)

А теперь можно написать функцию оценки всех данных в загрузчике:

In [ ]:
# TODO
def evaluate_loader(model, loader, device):
    metrics = {
        "accuracy": []
    }

    model.eval()
    for imgs_batch, labels_batch in loader:
        labels_batch = labels_batch.to(device)
        
    return metrics

In [ ]:
# TEST
config = {
    "lr": 0.01,
    # Для проверки - 2 эпохи
    "epochs": 2,
    "batch_size": 32,

    "train_dirpath": train_dirpath,
    "valid_dirpath": valid_dirpath,
    "test_dirpath": test_dirpath,
}

_, val_loader, _ = prepare_loaders(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MyFirstConvNet()
model.to(device)
epoch_metrics = evaluate_loader(model, val_loader, device)

# Усредним точности на батчах
epoch_mean_accuracy = np.mean(epoch_metrics["accuracy"])

np.testing.assert_almost_equal(epoch_mean_accuracy, 0.1, decimal=1)

Необученная модель должна давать показатель порядка 10% (0.1), что соответствует случайным предсказаниям 1 из 10 классов.

> ⚠️ Как видите, в оценке предсказаний модели нет ничего сложного. По сути, с модели приходят сырые степени уверенности, которые всего лишь нужно преобразовать в вектор предсказаний (одномерный) и дальше можно пользоваться всем функционалом оценки из sklearn!

Превосходно! Мы теперь можем ещё и оценивать метрику, которая более понятно описывает работу модели на каждой эпохе! Осталось теперь только запустить обучение и посмотреть на результаты!

## Пора учить по полной

Давайте выстроим весь наш цикл и постараемся обучить модель с учетом оценки Loss на выборке для обучения и Loss + accuracy на выборке валидации. Тестовая выборка остается нетронутой!

In [ ]:
config = {
    "lr": 0.01,
    "epochs": 50,
    # Возьмем побольше
    "batch_size": 128,

    "train_dirpath": train_dirpath,
    "valid_dirpath": valid_dirpath,
    "test_dirpath": test_dirpath,
}

torch.manual_seed(RANDOM_STATE)
train_loader, val_loader, test_loader = prepare_loaders(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyFirstConvNet()
model.to(device)

loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=config["lr"])

history = {
    "train_loss": [],
    "val_loss": [],
    "accuracy": []
}

for epoch in range(config["epochs"]):
    epoch_train_history = train_epoch(model, train_loader, loss_op, optim, device)
    epoch_val_history = valid_epoch(model, val_loader, loss_op, device)
    epoch_metrics = evaluate_loader(model, val_loader, device)

    # В историю обучения попадает среднее по эпохе значение
    epoch_train_loss = np.mean(epoch_train_history["loss"])
    epoch_val_loss = np.mean(epoch_val_history["loss"])
    epoch_accuracy = np.mean(epoch_metrics["accuracy"])

    history["train_loss"].append(epoch_train_loss)
    history["val_loss"].append(epoch_val_loss)
    history["accuracy"].append(epoch_accuracy)

    print(f"Epoch {epoch}:")
    print(f"  Train loss: {epoch_train_loss}")
    print(f"  Valid loss: {epoch_val_loss}")
    print(f"  Accuracy: {epoch_accuracy}")

In [ ]:
plt.figure(figsize=[20,10])

plt.subplot(121)
plt.plot(history["train_loss"], label="Train")
plt.plot(history["val_loss"], label="Validation")
plt.title("Loss")
plt.legend()
plt.grid()

plt.subplot(122)
plt.plot(history["accuracy"])
plt.title("Accuracy")
plt.grid()

plt.show()

Процесс не быстрый, но всё-таки это уже глубокое обучение! Некоторые сети могут учиться неделями или месяцами без перерыва!

Смотря на графики, как вы считаете, хватило ли 50 эпох, чтобы полностью обучиться модели? Попробуйте поменять параметры (размер батча, коэффициент обучения, количество эпох) и получить лучшие показатели модели!

> Подумайте, как в данной ситуации понять, что случилось переобучение/недообучение (overfit/underfit)?

## Оценка модели на тесте, "слоеном тесте" =)

После того, как найдена наилучшая модель (вот для чего нужна выборка валидации), следующим этапом является оценка модели на тестовой выборке. Так мы поймём, не слишком ли модель заточена под те данные, которые были использованы при обучении и валидации.

> Этот шаг также может показать, что тест координально отличается от остальных выборок, хотя он должен быть самым аккуратно подготовленным, так как на нём производится окончательная оценка!


In [ ]:
# Здесь просто подкладываем модель (лучшую) и загрузчик данных теста
epoch_metrics = evaluate_loader(model, test_loader, device)

epoch_mean_accuracy = np.mean(epoch_metrics["accuracy"])
print(f"Test acuracy: {epoch_mean_accuracy}")

Значения показателей на выборке для тестирования могут отличаться от показателей на выборке валидации, но главное, чтобы не было слишком больших отличий!

В этот момент мы можем закончить работу с моделью и сказать, что модель обучена хорошо (если у вас показатель на тесте выше 90%), но остался очень важный шаг - анализ ошибок!

## Анализ ошибок (вместе с визуальной оценкой)

Этот этап является не менее важным, чем обучение, так как модель никогда не будет давать 100% результат на больших данных (иначе слишком подозрительно и надо всё ещё раз проверить). Поэтому мы должны понять, в каких ситуация ошибается модель и можем ли мы исправить данные/улучшить модель?

Перво-наперво, нужно построить матрицу ошибок, то есть получить полный вектор предсказаний и полный вектор разметки для построения. Пишем!

In [ ]:
y_true = []
y_pred = []
# Сразу с созданием вектора предсказаний соберем инфу об ошибках
err_img_triplets = []

model.eval()
for imgs_batch, labels_batch in test_loader:
    imgs_batch = imgs_batch.to(device)
    labels_batch = labels_batch.to(device)
    
    pred = model(imgs_batch)
    _, pred_labels = torch.max(pred, dim=1)
    
    for i in range(imgs_batch.shape[0]):
        y_true.append(labels_batch[i].item())
        y_pred.append(pred_labels[i].item())

        if y_true[-1] != y_pred[-1]:
            err_img_triplets.append(
                (imgs_batch[i].cpu(), y_true[-1], y_pred[-1])
            )

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mtrx = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_mtrx, annot=True, fmt="d")
plt.xlabel("Predicted")
plt.ylabel("Ground Truth")
plt.show()

Видим, что в большинстве случаев предсказания корректны, но есть промахи, когда цифра 4 принята как 9, цифра 7 за 9, а также путается 3 и 8. Согласитесь, в некоторых случаях ошибки логичны, но важно взглянуть на них визуально!

Теперь построим отчёт классификации:

In [ ]:
from sklearn.metrics import classification_report

rep = classification_report(y_true, y_pred, digits=4)
print(rep)

И наконец взглянем на наши ошибки!

In [ ]:
plt.figure(figsize=[25, 10])
subplot = 1

for img_tnsr, true_label, pred_label in err_img_triplets[:5]:
    img = img_tnsr.permute(1, 2, 0).mul(255).numpy().astype(np.uint8)

    plt.subplot(1, 5, subplot)
    subplot += 1

    plt.imshow(img[..., 0], cmap="gray")
    plt.xticks([])
    plt.yticks([])
    plt.title(f"True: {true_label}, predicted: {pred_label}")

plt.show()

Видите, в одном случае вообще цифра похожа на 9, что и предсказывает модель, но разметка говорит, что это 3! Здесь либо разметка содержит ошибку, либо так и выглядит.

Визуальный анализ полезен тем, что позволяет понять, на каких случаях ошибается модель и насколько случаи выпадают из стандартных кейсов.

## Что мне делать со своими картинками?

Если вы хотите работать со своими изображениями, то необходимо написать класс, который будет выполнять предсказание по конкретной картинке. В данном случае сделать это несложно, поэтому разбираемся!

Возьмем для примера картинку из тестового набора (без разметки):

In [ ]:
image_path = Path("mnist_dataset") / "testSet" / "testSet" / "img_8.jpg"

vis_eval_image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)

Теперь напишем класс, который будет выполнять пресказание, на вход нужно лишь подать картинку:

In [ ]:
# TODO - реализуйте класс для предсказания цифры по картинке
class ImageInference:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        # Переведите модель на устройство

    def infer_image(self, image: np.ndarray) -> int:
        # Переведите модель в режим inference (.eval())

        # Проверить, картинка (28, 28) или (28, 28, 1)
        #   В первом случае добавить размерность

        # Преобразовать изображение в тензор с подготовкой в формате CHW
        # Преобразовать в батч: CHW -> NCHW

        # Не забывайте, что применение данных вне фреймворка PyTorch 
        #   требует возврата на CPU методом .cpu()

        # Вернуть нужно предсказание (название класса), не степени уверенности!
        raise NotImplementedError()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inferencer = ImageInference(model, device)

pred_label = inferencer.infer_image(vis_eval_image)

plt.imshow(vis_eval_image, cmap="gray")
plt.title(f"Predicted: {pred_label}")
plt.show()

Если вы видите цифру 3 - великолепно! Значит все загружено правильно, цифра отобразилась и над ней должно появиться предсказание модели!

## Рекомендации

Что же, основы основ мы посмотрели и в дальнейший путь хочется дать пару напутствий:
- Всегда проверяйте и отображайте размеры тензоров - из-за их сложности (больше двух размерностей) они являются одной из первопричин багов, а так как есть такие умные механизмы типа "broadcasting", то можно совершить ошибку, но всё будет работать - только неправильно работать =)
- Не скупитесь на написание кода для визуализации - в работе с изображениями это один из наиболее полезных способов пониманимя данных
- "Своя архитектура - последнее дело" - на сегодняшний дел разработана огромная куча различныъх архитектур свёрточных нейросетей, каждая со своими особенностями (плюсами и минусами); не пытайтесь написать своё - лучше воспользуйтесь готовыми, а уж если ни одна не подошла (верится с трудом), то можно браться за разработку своей "супер-крутой" архитектуры

## Вопросы (заметки)

- Как сделать так, чтобы можно было задать количество предсказываемых классов модели? Например, сейчас, если поменяется количество предсказываемых классов - нужно менять код модели.
- Как сделать так, чтобы модель учитывала размер входа при формировании полносвязного слоя? Например, сейчас, если поменяется размер входа на 32х32 - нужно менять код модели.
- Зачем мы написали датасет, который каждый раз загружает картинки с диска? Почему нельзя разом загрузить все изображения и хранить в списке?
